# Electric Vehicle Population Data Analysis
# Data Science Fall 2025 - Assignment 3

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, classification_report, silhouette_score
import warnings
warnings.filterwarnings('ignore')

# Set style for better visualizations
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# 1. DATA LOADING AND INITIAL EXPLORATION

In [2]:
import pandas as pd

df = pd.read_csv(r'C:\Users\parid\Documents\Fall 2025\Data Science\Assignment 3\Electric_Vehicle_Population_Data.csv')

# Quick check
print("Dataset loaded successfully!")
print(df.shape)
df.head()

df.head()


Dataset loaded successfully!
(264628, 17)


,VIN (1-10),County,City,State,Postal Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,2020 Census Tract
0,WA1E2AFY8R,Thurston,Olympia,WA,98512.0,2024,AUDI,Q5 E,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,23.0,0.0,22.0,263239938,POINT (-122.90787 46.9461),PUGET SOUND ENERGY INC,5.306701e+10
1,WAUUPBFF4J,Yakima,Wapato,WA,98951.0,2018,AUDI,A3,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,16.0,0.0,15.0,318160860,POINT (-120.42083 46.44779),PACIFICORP,5.307794e+10
2,1N4AZ0CP0F,King,Seattle,WA,98125.0,2015,NISSAN,LEAF,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,84.0,0.0,46.0,184963586,POINT (-122.30253 47.72656),CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),5.303300e+10
3,WA1VAAGE5K,King,Kent,WA,98031.0,2019,AUDI,E-TRON,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,204.0,0.0,11.0,259426821,POINT (-122.17743 47.41185),PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),5.303303e+10
4,7SAXCAE57N,Snohomish,Bothell,WA,98021.0,2022,TESLA,MODEL X,Battery Electric Vehicle (BEV),Eligibility unknown as battery range has not b...,0.0,0.0,1.0,208182236,POINT (-122.18384 47.8031),PUGET SOUND ENERGY INC,5.306105e+10


# 2. DATA CLEANING AND PREPROCESSING

In [3]:
def clean_data(df):

    print("\n--- DATA CLEANING ---")

    # Display initial info
    print(f"\nInitial shape: {df.shape}")
    print(f"Missing values:\n{df.isnull().sum()}")

    # Create a copy
    df_clean = df.copy()

    # Handle missing values
    # Drop rows with missing critical values
    critical_cols = ['Model Year', 'Make', 'Model', 'Electric Vehicle Type']
    df_clean = df_clean.dropna(subset=critical_cols)

    # Fill numeric columns with median
    numeric_cols = df_clean.select_dtypes(include=[np.number]).columns
    for col in numeric_cols:
        if df_clean[col].isnull().sum() > 0:
            df_clean[col].fillna(df_clean[col].median(), inplace=True)

    # Fill categorical columns with mode
    categorical_cols = df_clean.select_dtypes(include=['object']).columns
    for col in categorical_cols:
        if df_clean[col].isnull().sum() > 0:
            df_clean[col].fillna(df_clean[col].mode()[0], inplace=True)

    print(f"\nCleaned shape: {df_clean.shape}")
    print(f"Rows removed: {df.shape[0] - df_clean.shape[0]}")

    return df_clean

# 3. EXPLORATORY DATA ANALYSIS (EDA)

In [4]:
def perform_eda(df):

    print("\n--- EXPLORATORY DATA ANALYSIS ---")

    # Create figure directory
    import os
    if not os.path.exists('figures'):
        os.makedirs('figures')

    # 1. Distribution of Electric Vehicle Types
    plt.figure(figsize=(10, 6))
    if 'Electric Vehicle Type' in df.columns:
        ev_counts = df['Electric Vehicle Type'].value_counts()
        plt.bar(ev_counts.index, ev_counts.values, color=['#2ecc71', '#3498db'])
        plt.title('Distribution of Electric Vehicle Types', fontsize=16, fontweight='bold')
        plt.xlabel('Vehicle Type', fontsize=12)
        plt.ylabel('Count', fontsize=12)
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.savefig('figures/ev_type_distribution.png', dpi=300, bbox_inches='tight')
        plt.close()
        print("✓ EV Type Distribution plot saved")

    # 2. Top 10 EV Makes
    plt.figure(figsize=(12, 6))
    if 'Make' in df.columns:
        top_makes = df['Make'].value_counts().head(10)
        plt.barh(top_makes.index, top_makes.values, color='steelblue')
        plt.title('Top 10 Electric Vehicle Manufacturers', fontsize=16, fontweight='bold')
        plt.xlabel('Number of Vehicles', fontsize=12)
        plt.ylabel('Manufacturer', fontsize=12)
        plt.tight_layout()
        plt.savefig('figures/top_manufacturers.png', dpi=300, bbox_inches='tight')
        plt.close()
        print("✓ Top Manufacturers plot saved")

    # 3. EV Adoption Over Years
    plt.figure(figsize=(14, 6))
    if 'Model Year' in df.columns:
        yearly_counts = df['Model Year'].value_counts().sort_index()
        plt.plot(yearly_counts.index, yearly_counts.values, marker='o',
                linewidth=2, markersize=8, color='#e74c3c')
        plt.fill_between(yearly_counts.index, yearly_counts.values, alpha=0.3, color='#e74c3c')
        plt.title('Electric Vehicle Adoption Trend Over Years', fontsize=16, fontweight='bold')
        plt.xlabel('Model Year', fontsize=12)
        plt.ylabel('Number of Vehicles', fontsize=12)
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.savefig('figures/adoption_trend.png', dpi=300, bbox_inches='tight')
        plt.close()
        print("✓ Adoption Trend plot saved")

    # 4. Electric Range Distribution
    plt.figure(figsize=(12, 6))
    if 'Electric Range' in df.columns:
        range_data = df[df['Electric Range'] > 0]['Electric Range']
        plt.hist(range_data, bins=50, color='#9b59b6', edgecolor='black', alpha=0.7)
        plt.axvline(range_data.mean(), color='red', linestyle='--',
                   linewidth=2, label=f'Mean: {range_data.mean():.1f} miles')
        plt.axvline(range_data.median(), color='green', linestyle='--',
                   linewidth=2, label=f'Median: {range_data.median():.1f} miles')
        plt.title('Distribution of Electric Vehicle Range', fontsize=16, fontweight='bold')
        plt.xlabel('Electric Range (miles)', fontsize=12)
        plt.ylabel('Frequency', fontsize=12)
        plt.legend()
        plt.tight_layout()
        plt.savefig('figures/range_distribution.png', dpi=300, bbox_inches='tight')
        plt.close()
        print("✓ Range Distribution plot saved")

    # 5. Geographic Distribution by County
    plt.figure(figsize=(14, 8))
    if 'County' in df.columns:
        top_counties = df['County'].value_counts().head(15)
        plt.barh(range(len(top_counties)), top_counties.values, color='coral')
        plt.yticks(range(len(top_counties)), top_counties.index)
        plt.title('Top 15 Counties by EV Population', fontsize=16, fontweight='bold')
        plt.xlabel('Number of Vehicles', fontsize=12)
        plt.ylabel('County', fontsize=12)
        plt.tight_layout()
        plt.savefig('figures/county_distribution.png', dpi=300, bbox_inches='tight')
        plt.close()
        print("✓ County Distribution plot saved")

    # 6. Correlation Heatmap for numeric variables
    plt.figure(figsize=(10, 8))
    numeric_df = df.select_dtypes(include=[np.number])
    if numeric_df.shape[1] > 1:
        corr_matrix = numeric_df.corr()
        sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm',
                   center=0, square=True, linewidths=1)
        plt.title('Correlation Matrix of Numeric Variables', fontsize=16, fontweight='bold')
        plt.tight_layout()
        plt.savefig('figures/correlation_heatmap.png', dpi=300, bbox_inches='tight')
        plt.close()
        print("✓ Correlation Heatmap saved")

    # Summary Statistics
    print("\n--- SUMMARY STATISTICS ---")
    print(df.describe())

    return df


# 4. HYPOTHESIS 1: PREDICT ELECTRIC RANGE USING REGRESSION

In [5]:
def hypothesis_1_regression(df):

    print("\n" + "=" * 80)
    print("HYPOTHESIS 1: PREDICTING ELECTRIC RANGE")
    print("=" * 80)

    # Prepare data
    df_reg = df.copy()

    # Filter for Battery Electric Vehicles with valid range
    if 'Electric Vehicle Type' in df_reg.columns:
        df_reg = df_reg[df_reg['Electric Vehicle Type'] == 'Battery Electric Vehicle (BEV)']
    df_reg = df_reg[df_reg['Electric Range'] > 0]

    # Select features
    features = ['Model Year']
    if 'Base MSRP' in df_reg.columns:
        df_reg = df_reg[df_reg['Base MSRP'] > 0]
        features.append('Base MSRP')

    # Encode Make
    if 'Make' in df_reg.columns:
        le_make = LabelEncoder()
        df_reg['Make_encoded'] = le_make.fit_transform(df_reg['Make'])
        features.append('Make_encoded')

    X = df_reg[features]
    y = df_reg['Electric Range']

    # Split data
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    # Train Linear Regression Model
    lr_model = LinearRegression()
    lr_model.fit(X_train, y_train)

    # Predictions
    y_pred_train = lr_model.predict(X_train)
    y_pred_test = lr_model.predict(X_test)

    # Evaluation
    train_r2 = r2_score(y_train, y_pred_train)
    test_r2 = r2_score(y_test, y_pred_test)
    train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
    test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))

    print(f"\n--- LINEAR REGRESSION RESULTS ---")
    print(f"Training R² Score: {train_r2:.4f}")
    print(f"Testing R² Score: {test_r2:.4f}")
    print(f"Training RMSE: {train_rmse:.2f} miles")
    print(f"Testing RMSE: {test_rmse:.2f} miles")

    # Feature importance
    print(f"\n--- FEATURE COEFFICIENTS ---")
    for feat, coef in zip(features, lr_model.coef_):
        print(f"{feat}: {coef:.4f}")
    print(f"Intercept: {lr_model.intercept_:.4f}")

    # Visualization: Actual vs Predicted
    plt.figure(figsize=(10, 6))
    plt.scatter(y_test, y_pred_test, alpha=0.5, s=50)
    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()],
            'r--', lw=2, label='Perfect Prediction')
    plt.xlabel('Actual Electric Range (miles)', fontsize=12)
    plt.ylabel('Predicted Electric Range (miles)', fontsize=12)
    plt.title('Linear Regression: Actual vs Predicted Range', fontsize=16, fontweight='bold')
    plt.legend()
    plt.tight_layout()
    plt.savefig('figures/regression_actual_vs_predicted.png', dpi=300, bbox_inches='tight')
    plt.close()
    print("✓ Regression visualization saved")

    return lr_model, test_r2, test_rmse

# 5. HYPOTHESIS 2: CLASSIFY CAFV ELIGIBILITY

In [6]:

def hypothesis_2_classification(df):

    print("\n" + "=" * 80)
    print("HYPOTHESIS 2: CLASSIFYING CAFV ELIGIBILITY")
    print("=" * 80)

    # Prepare data
    df_clf = df.copy()

    # Create binary target (Eligible vs Not Eligible)
    if 'Clean Alternative Fuel Vehicle (CAFV) Eligibility' in df_clf.columns:
        df_clf['CAFV_Binary'] = df_clf['Clean Alternative Fuel Vehicle (CAFV) Eligibility'].apply(
            lambda x: 1 if 'Eligible' in str(x) else 0
        )

    # Select features
    features = ['Model Year', 'Electric Range']

    # Encode Make and Model
    if 'Make' in df_clf.columns:
        le_make = LabelEncoder()
        df_clf['Make_encoded'] = le_make.fit_transform(df_clf['Make'])
        features.append('Make_encoded')

    if 'Electric Vehicle Type' in df_clf.columns:
        le_type = LabelEncoder()
        df_clf['EV_Type_encoded'] = le_type.fit_transform(df_clf['Electric Vehicle Type'])
        features.append('EV_Type_encoded')

    # Remove rows with missing values
    df_clf = df_clf.dropna(subset=features + ['CAFV_Binary'])

    X = df_clf[features]
    y = df_clf['CAFV_Binary']

    # Split data
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )

    # Train Random Forest Classifier
    rf_model = RandomForestClassifier(n_estimators=100, random_state=42, max_depth=10)
    rf_model.fit(X_train, y_train)

    # Predictions
    y_pred_train = rf_model.predict(X_train)
    y_pred_test = rf_model.predict(X_test)

    # Evaluation
    train_accuracy = rf_model.score(X_train, y_train)
    test_accuracy = rf_model.score(X_test, y_test)

    print(f"\n--- RANDOM FOREST CLASSIFICATION RESULTS ---")
    print(f"Training Accuracy: {train_accuracy:.4f}")
    print(f"Testing Accuracy: {test_accuracy:.4f}")

    print(f"\n--- CLASSIFICATION REPORT (Test Set) ---")
    print(classification_report(y_test, y_pred_test,
                               target_names=['Not Eligible', 'Eligible']))

    # Feature importance
    print(f"\n--- FEATURE IMPORTANCE ---")
    feature_importance = pd.DataFrame({
        'Feature': features,
        'Importance': rf_model.feature_importances_
    }).sort_values('Importance', ascending=False)
    print(feature_importance)

    # Visualization: Feature Importance
    plt.figure(figsize=(10, 6))
    plt.barh(feature_importance['Feature'], feature_importance['Importance'], color='teal')
    plt.xlabel('Importance', fontsize=12)
    plt.ylabel('Feature', fontsize=12)
    plt.title('Random Forest: Feature Importance for CAFV Eligibility',
             fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.savefig('figures/classification_feature_importance.png', dpi=300, bbox_inches='tight')
    plt.close()
    print("Classification visualization saved successfully")

    return rf_model, test_accuracy

In [7]:
pip install scikit-learn-intelex

   ---------------------------------------- 0.0/3.1 MB ? eta -:--:--
   ---------------------------------------- 3.1/3.1 MB 18.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/82.2 MB ? eta -:--:--
   ----- ---------------------------------- 12.3/82.2 MB 59.1 MB/s eta 0:00:02
   ----------- ---------------------------- 23.9/82.2 MB 55.9 MB/s eta 0:00:02
   ----------------- ---------------------- 36.2/82.2 MB 56.0 MB/s eta 0:00:01
   ---------------------- ----------------- 45.9/82.2 MB 53.0 MB/s eta 0:00:01
   --------------------------- ------------ 56.9/82.2 MB 52.5 MB/s eta 0:00:01
   ------------------------------- -------- 65.3/82.2 MB 50.1 MB/s eta 0:00:01
   ------------------------------------- -- 76.8/82.2 MB 51.0 MB/s eta 0:00:01
   ---------------------------------------  82.1/82.2 MB 50.8 MB/s eta 0:00:01
   ---------------------------------------  82.1/82.2 MB 50.8 MB/s eta 0:00:01
   ---------------------------------------  82.1/82.2 MB 50.8 MB/s eta 0:

In [8]:
from sklearnex import patch_sklearn
patch_sklearn()

Extension for Scikit-learn* enabled (https://github.com/uxlfoundation/scikit-learn-intelex)


#  CLUSTERING ANALYSIS

In [9]:
# !pip install scikit-learn-intelex
from sklearnex import patch_sklearn
patch_sklearn()

# Import all other necessary libraries
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt


Extension for Scikit-learn* enabled (https://github.com/uxlfoundation/scikit-learn-intelex)


In [10]:
def clustering_analysis(df, silhouette_sample_size=50000):
   
    print("\n" + "=" * 80)
    print("CLUSTERING ANALYSIS: VEHICLE SEGMENTATION")
    print("=" * 80)

    # Prepare data
    df_cluster = df.copy()

    # Select features for clustering
    cluster_features = ['Model Year', 'Electric Range']

    # Remove rows with missing values
    df_cluster = df_cluster.dropna(subset=cluster_features)
    df_cluster = df_cluster[df_cluster['Electric Range'] > 0]

    X_cluster = df_cluster[cluster_features]

    # Check if data is empty after filtering
    if X_cluster.empty:
        print("No data available for clustering after filtering.")
        return None, 0

    print(f"Clustering on {len(X_cluster)} data points...")

    # Standardize features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_cluster)

    # Elbow method to find optimal k
    inertias = []
    silhouette_scores = []
    K_range = range(2, 11)

    # Determine the actual sample size to use (can't be larger than dataset)
    n_samples = X_scaled.shape[0]
    if n_samples <= silhouette_sample_size:
        print("Dataset is small, using full data for silhouette score.")
        actual_sample_size = None
    else:
        print(f"Using sample_size={silhouette_sample_size} for silhouette score.")
        actual_sample_size = silhouette_sample_size


    for k in K_range:
        print(f"... Processing k={k}")
        kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
        kmeans.fit(X_scaled)
        inertias.append(kmeans.inertia_)


        labels = kmeans.labels_
        silhouette_scores.append(silhouette_score(
            X_scaled,
            labels,
            sample_size=actual_sample_size,
            random_state=42
        ))
        # ------------------------------------

    # Plot elbow curve
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

    ax1.plot(K_range, inertias, marker='o', linewidth=2, markersize=8)
    ax1.set_xlabel('Number of Clusters (k)', fontsize=12)
    ax1.set_ylabel('Inertia', fontsize=12)
    ax1.set_title('Elbow Method: Optimal k', fontsize=14, fontweight='bold')
    ax1.grid(True, alpha=0.3)

    ax2.plot(K_range, silhouette_scores, marker='o', linewidth=2,
             markersize=8, color='orange')
    ax2.set_xlabel('Number of Clusters (k)', fontsize=12)
    ax2.set_ylabel('Silhouette Score', fontsize=12)
    ax2.set_title('Silhouette Score by k', fontsize=14, fontweight='bold')
    ax2.grid(True, alpha=0.3)

    plt.tight_layout()
    # Create figures directory if it doesn't exist (optional but good practice)
    # import os
    # os.makedirs('figures', exist_ok=True)
    plt.savefig('figures/clustering_elbow_method.png', dpi=300, bbox_inches='tight')
    plt.close()
    print("✓ Elbow method plot saved")


    optimal_k = 4

    kmeans_final = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
    df_cluster['Cluster'] = kmeans_final.fit_predict(X_scaled)

    print(f"\n--- K-MEANS CLUSTERING RESULTS (k={optimal_k}) ---")


    final_score = silhouette_score(
        X_scaled,
        kmeans_final.labels_,
        sample_size=actual_sample_size,
        random_state=42
    )
    print(f"Silhouette Score (sampled): {final_score:.4f}")
    print(f"Inertia: {kmeans_final.inertia_:.2f}")

    # Cluster distribution
    print(f"\n--- CLUSTER DISTRIBUTION ---")
    print(df_cluster['Cluster'].value_counts().sort_index())

    # Visualize clusters
    plt.figure(figsize=(12, 8))
    scatter = plt.scatter(df_cluster['Model Year'], df_cluster['Electric Range'],
                            c=df_cluster['Cluster'], cmap='viridis',
                            s=50, alpha=0.6, edgecolors='black', linewidth=0.5)
    plt.xlabel('Model Year', fontsize=12)
    plt.ylabel('Electric Range (miles)', fontsize=12)
    plt.title(f'K-Means Clustering: Vehicle Segmentation (k={optimal_k})',
                fontsize=16, fontweight='bold')
    plt.colorbar(scatter, label='Cluster')
    plt.tight_layout()
    plt.savefig('figures/clustering_visualization.png', dpi=300, bbox_inches='tight')
    plt.close()
    print("✓ Clustering visualization saved")

    # Cluster characteristics
    print(f"\n--- CLUSTER CHARACTERISTICS ---")
    cluster_summary = df_cluster.groupby('Cluster')[cluster_features].mean()
    print(cluster_summary)

    return kmeans_final, optimal_k


# 7. MAIN EXECUTION FUNCTION

In [12]:
def main():
    """Main execution function"""

    print("\n" + "=" * 80)
    print("STARTING ELECTRIC VEHICLE DATA ANALYSIS")
    print("=" * 80)

    df_clean = clean_data(df)
    df_clean = perform_eda(df_clean)

    # Run analyses
    lr_model, r2, rmse = hypothesis_1_regression(df_clean)
    rf_model, accuracy = hypothesis_2_classification(df_clean)
    kmeans_model, k = clustering_analysis(df_clean)

    print("\n" + "=" * 80)
    print("ANALYSIS COMPLETE!")
    print("All figures saved in 'figures/' directory")
    print("=" * 80)

if __name__ == "__main__":
    main()

    # Instructions for running:
    print("\n" + "=" * 80)
    print("TO RUN THIS CODE:")
    print("1. Save this file as 'ev_analysis.py'")
    print("2. Ensure your CSV file is in the same directory")
    print("3. Update the file path in load_data() function")
    print("4. Run: python ev_analysis.py")
    print("=" * 80)




STARTING ELECTRIC VEHICLE DATA ANALYSIS

--- DATA CLEANING ---

Initial shape: (264628, 17)
Missing values:
VIN (1-10)                                             0
County                                                 9
City                                                   9
State                                                  0
Postal Code                                            9
Model Year                                             0
Make                                                   0
Model                                                  0
Electric Vehicle Type                                  0
Clean Alternative Fuel Vehicle (CAFV) Eligibility      0
Electric Range                                         4
Base MSRP                                              4
Legislative District                                 659
DOL Vehicle ID                                         0
Vehicle Location                                      17
Electric Utility                    